### 0. Setup

Import the libraries we need.

In [ ]:
! pip install ISLP
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.compat import lzip
import seaborn as sns
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

### 1. Import data

Load the data, and explore the relationship among all the attributes.


### Overview
The **Boston Housing Dataset** contains information on **506 housing units** in the suburbs of Boston, with **13 features** used to predict the median value of homes in different neighborhoods. Each sample in the dataset represents the statistical information of a particular area.

### Features
The dataset typically includes the following variables:

1. **CRIM**: Per capita crime rate by town
2. **ZN**: Proportion of residential land zoned for lots over 25,000 sq. ft.
3. **INDUS**: Proportion of non-retail business acres per town
4. **CHAS**: Charles River dummy variable (1 if tract bounds river; 0 otherwise)
5. **NOX**: Nitric oxides concentration (parts per 10 million)
6. **RM**: Average number of rooms per dwelling
7. **AGE**: Proportion of owner-occupied units built prior to 1940
8. **DIS**: Weighted distances to five Boston employment centers
9. **RAD**: Index of accessibility to radial highways
10. **TAX**: Full-value property tax rate per \$10,000
11. **PTRATIO**: Pupil-teacher ratio by town
12. **B**: Calculation \( B = 1000(Bk - 0.63)^2 \) where Bk is the proportion of Black residents
13. **LSTAT**: Percentage of the population considered lower status
14. **MEDV**: Median value of owner-occupied homes (in \$1000s)



In [ ]:
Boston = load_data("Boston")
print(Boston.shape)
print(Boston.columns)

In [ ]:
Boston.isna().any()

In [ ]:
Boston[['crim', 'zn','indus', 'chas', 'nox']]

In [ ]:
sns.pairplot(Boston[['crim', 'zn','indus', 'chas', 'nox']])

### 2. Modeling (Autocorrelation and Normality)

In [ ]:
terms0 = Boston.columns.drop(['indus', 'age', 'medv'])
X0 = MS(terms0).fit_transform(Boston)
y0 = Boston['medv']
model0 = sm.OLS(y0, X0)
results0 = model0.fit()
results0.summary()

### Condition Number for Detecting Collinearity

The condition number is commonly used to detect collinearity in regression analysis or other linear algebra applications. Collinearity refers to the situation where two or more independent variables in a model are highly correlated, which can lead to instability and inaccurate coefficient estimates. Here’s how the condition number is used to assess collinearity:

### Condition Number and Collinearity
In multiple linear regression, the condition number $\kappa(X) $ of the design matrix $X $ can be used to assess collinearity. The condition number is calculated as:

$$
\kappa(X) = \frac{\sigma_{\text{max}}}{\sigma_{\text{min}}}
$$
where:
- $\sigma_{\text{max}} $ is the largest singular value of matrix $X $
- $\sigma_{\text{min}} $ is the smallest singular value of matrix $X $

### Collinearity Assessment Criteria
The condition number helps in identifying the degree of collinearity based on the following thresholds:

- **1 ≤ $\kappa(X) $ < 10**: Little to no collinearity, and the model is usually stable.
- **10 ≤ $\kappa(X)$ < 30**: Moderate collinearity is present, which may affect the model estimation to some extent.
- **$\kappa(X) \geq 30$**: Severe collinearity is present, indicating that the model estimates are likely very unstable, and coefficient estimates may be unreliable.

### Key Points
- **High Condition Number Indicates Collinearity**: A large condition number suggests that the columns of matrix $X $ are highly correlated, leading to collinearity. This makes it difficult for the model to accurately estimate the coefficients due to overlapping information among the variables.
- **Standardizing Data**: It is common to standardize data before calculating the condition number to eliminate the influence of different scales or units on the results.

By evaluating the condition number, you can effectively detect multicollinearity issues in a model and take appropriate actions, such as removing correlated variables or applying techniques like principal component regression, to address the impact of collinearity.



### Interaction Terms

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True,include_bias = False)
terms0 = Boston.columns.drop(['indus', 'age', 'medv'])
y = Boston['medv']
X = poly.fit_transform(Boston)
Xb = sm.add_constant(X)
mod = sm.OLS(y, Xb)
res = mod.fit()
res.summary()

### 3. High-leverage points

### 1. Code Explanation
https://www.statsmodels.org/stable/generated/statsmodels.graphics.regressionplots.influence_plot.html

external:bool

Whether to use externally or internally studentized residuals. It is recommended to leave external as True (default).

external residual represents the deleted residual.

### 2. What the Plot Represents
- Row labels for the observations in which the leverage, measured by the diagonal of the hat matrix, is high or the residuals are large, as the combination of large residuals and a high influence value indicates an influence point. The value of large residuals can be controlled using the alpha parameter. Large leverage points are identified as hat_i > 2 * (df_model + 1)/nobs.


- Cook's Distance measures the influence of each data point on the overall fit of the model; a larger value indicates that the data point has a more substantial impact on the model’s estimates.

- While leverage (the X-axis) indicates how much a data point differs in the independent variable space (i.e., how far it is from other points), the size of the points is determined by Cook's Distance, which takes into account both leverage and the residuals.



In [ ]:
fig = sm.graphics.influence_plot(results0,criterion='cooks',)
fig.tight_layout(pad=1.0)

In [ ]:
df = Boston[terms0.append(pd.Index(["medv"]))]
print("the scale of raw data:", df.shape)
# obtain Cook's distance
lm_cooksd = results0.get_influence().cooks_distance[0]

# get length of df to obtain n
n = len(Boston["medv"])

# calculate critical d
critical_d = 4/n
print('Critical Cooks distance:', critical_d)

# identification of potential outliers with leverage
out_d = lm_cooksd > critical_d

# output potential outliers with leverage
print(df.index[out_d], "\n",
    lm_cooksd[out_d])
print("lenth:", len(lm_cooksd[out_d]))
subset = ~df.index.isin(df.index[out_d].tolist())
Boston = Boston.iloc[subset] #remove outliers and fit again

In [ ]:
terms0 = Boston.columns.drop(['indus', 'age', 'medv'])
X0 = MS(terms0).fit_transform(Boston)
y0 = Boston['medv']
model0 = sm.OLS(y0, X0)
results0 = model0.fit()
results0.summary()

### 4. Heteroscedasticity

In [ ]:
# fitted values
model_fitted_y = results0.fittedvalues
df = Boston[terms0.append(pd.Index(["medv"]))]
#  Plot
plot = sns.residplot(x=model_fitted_y, y='medv', data=df, lowess=True,
                     scatter_kws={'alpha': 0.5},
                     line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

# Titel and labels
plot.set_title('Residuals vs Fitted')
plot.set_xlabel('Fitted values')
plot.set_ylabel('Residuals');

### Test for Heteroscedasticity

The Breusch-Pagan test is used to detect heteroscedasticity in a regression model.


- Lagrange multiplier statistic: A statistic used to test for heteroscedasticity.
- p-value: The p-value associated with the Lagrange multiplier statistic, indicating the significance level.
- f-value: The F-test statistic, another method of testing for heteroscedasticity.
- f p-value: The p-value corresponding to the F-test statistic.

In [ ]:
name = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
test = sm.stats.het_breuschpagan(results0.resid, results0.model.exog)
lzip(name, test)

### 5. Collinearity



In [ ]:
# Inspect correlation
# Calculate correlation using the default method ( "pearson")
df = Boston[terms0]
corr = df.corr()
# optimize aesthetics: generate mask for removing duplicate / unnecessary info
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
# Generate a custom diverging colormap as indicator for correlations:
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Plot
sns.heatmap(corr, mask=mask, cmap=cmap, annot=True,  square=True, annot_kws={"size": 12});

In [ ]:
terms0 = Boston.columns.drop(['indus', 'age', 'medv', 'rad']) #drop rad because of high correlation
X0 = MS(terms0).fit_transform(Boston)
y0 = Boston['medv']
model0 = sm.OLS(y0, X0)
results0 = model0.fit()
results0.summary() #Conditional Number decrease

 ### 6. Only focus on the independent variables which have linear relationship with dependent variable

In [ ]:
terms0 = Boston.columns.drop(['zn','crim','nox', 'indus', 'chas', 'age', 'rad', 'tax',
       'ptratio','medv'])
X0 = MS(terms0).fit_transform(Boston)
y0 = Boston['medv']
model0 = sm.OLS(y0, X0)
results0 = model0.fit()
results0.summary() #Conditional number pretty small

In [ ]:
df = Boston[terms0.append(pd.Index(["medv"]))]
print("the scale of raw data:", df.shape)
# obtain Cook's distance
lm_cooksd = results0.get_influence().cooks_distance[0]

# get length of df to obtain n
n = len(Boston["medv"])

# calculate critical d
critical_d = 4/n
print('Critical Cooks distance:', critical_d)

# identification of potential outliers with leverage
out_d = lm_cooksd > critical_d

# output potential outliers with leverage
print(df.index[out_d], "\n",
    lm_cooksd[out_d])
print("lenth:", len(lm_cooksd[out_d]))
subset = ~df.index.isin(df.index[out_d].tolist())
Boston = Boston.iloc[subset]
terms0 = Boston.columns.drop(['zn','crim','nox', 'indus', 'chas', 'age', 'rad', 'tax',
       'ptratio','medv'])
X0 = MS(terms0).fit_transform(Boston)
y0 = Boston['medv']
model0 = sm.OLS(y0, X0)
results0 = model0.fit()
results0.summary()

In [ ]:
name = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
test = sm.stats.het_breuschpagan(results0.resid, results0.model.exog)
lzip(name, test)

In [ ]:
sns.pairplot(Boston)